In [1]:
import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit
import json
 
# Data to be written
games = []
        
def request(msg, slp=0.5):
    # A wrapper to make robust https requests.
    
    # Want to get a status-code of 200
    status_code = 500  
    i = 4
    while status_code != 200:
        # Don't ping the server too often
        sleep(slp)  
        try:
            r = requests.get(msg)
            status_code = r.status_code
            i -= 1
            #if status_code != 200:
                #print('Server Error! Response Code %i. Retrying...' % (r.status_code))
            if i == 0:
                print('Server Error! Response Code %i. Retrying...' % (r.status_code))
                return 0
        except:
            print('An exception has occurred, probably a momentary loss of connection. Waiting one seconds...')
            sleep(1)
    return r


# Initialize a DF to hold all our scraped game info
df_all = pd.DataFrame(columns=['id', 'name', 'nrate', 'avg_rate', 'pic_url'])
min_nrate = 1e5
npage = 1

# Scrap successful pages in the results until we get down to games with < 1000 ratings each
while min_nrate > 5000:
    
    # Get full HTML for a specific page in the full listing of boardgames sorted by 
    r = request('https://boardgamegeek.com/browse/boardgame/page/%i?sort=numvoters&sortdir=desc' % (npage,))
    
    if r == 0:
        continue
    
    soup = BeautifulSoup(r.text, 'html.parser')    
    
    # Get list of all the rows (tags) in the list of games on this page
    table = soup.find_all('tr', attrs={'id': 'row_'})
    
    # DF to hold this pages results
    df = pd.DataFrame(columns=['id', 'name', 'nrate', 'avg_rate', 'pic_url'], index=range(len(table)))  
    
    # Loop through each row and pull out the info for that game
    for idx, row in enumerate(table):
        
        # Row may or may not start with a "boardgame rank" link, if YES then strip it
        links = row.find_all('a')
        if 'name' in links[0].attrs.keys():
            del links[0]
            
        # Get the relative URL for the specific game
        gamelink = links[1]
        
        # Get the game ID by parsing the relative URL
        gameid = int(gamelink['href'].split('/')[2])
        
        # Get the actual name of the game as the link contents
        gamename = gamelink.contents[0]
        
        # Get the URL for the game thumbnail
        imlink = links[0]
        thumbnail = imlink.contents[0]['src']
        
        ratings_str = row.find_all('td', attrs={'class': 'collection_bggrating'})[1].contents[0]
        avg_rate = float(''.join(ratings_str.split()))

        ratings_str = row.find_all('td', attrs={'class': 'collection_bggrating'})[2].contents[0]
        nratings = int(''.join(ratings_str.split()))

        df.iloc[idx, :] = [gameid, gamename, nratings, avg_rate, thumbnail]
    
    try:
        # The smallest number of ratings of any game on the page
        min_nrate = df['nrate'].min()
        print('Page %i scraped, minimum number of ratings was %i' % (npage, min_nrate))
    except:
        print('Page %i scraped, minimum number of ratings was NaN' %(npage,))
    
     # Concatenate the results of this page to the master dataframe
    df_all = pd.concat([df_all, df], axis=0)
    npage += 1
    
    # Keep the BGG server happy.
    sleep(0.5)
    

df = df_all.copy()

# Reset the index since we concatenated a bunch of DFs with the same index into one DF
df.reset_index(inplace=True, drop=True)

# Write the DF to .csv for future use
df.to_csv('bgg_gamelist.csv', index=False, encoding="utf-8")

csv = pd.read_csv('bgg_gamelist.csv')

w = 1
v = 1

for i in range(17,300):

    r = request('https://api.geekdo.com/xmlapi/boardgame/%i?marketplace=1' % (csv['id'][i],))
    
    if r == 0:
        continue
    
    soup = BeautifulSoup(r.text, "xml")
    
    #sales_offers = []
    #j = 0
    #
    #for attributes in soup.marketplacelistings:
    #    if j%2 == 1:
    #        offer = {
    #            'date'      : attributes.listdate.string,
    #            'currency'  : attributes.price['currency'],
    #            'price'     : attributes.price.string,
    #            'condition' : attributes.condition.string,
    #            'notes'     : attributes.notes.string
    #        }
    #        sales_offers.append(offer)
    #    j += 1
    
    tmp = soup.find_all('boardgamecategory')
    categories = []
    
    for category in tmp:
        categories.append(category.string)
    
    game = {
        'name'          : str(csv['name'][i]),
        'nrate'         : str(csv['nrate'][i]),
        'designer'      : soup.boardgamedesigner.string,
        'yearpublished' : soup.yearpublished.string,
        'minplayers'    : soup.minplayers.string,
        'maxplayers'    : soup.maxplayers.string,
        'playingtime'   : soup.playingtime.string,
        'minplaytime'   : soup.minplaytime.string,
        'maxplaytime'   : soup.maxplaytime.string,
        'categories'    : categories,
        'description'   : soup.description.string,
        'img'           : soup.image.string,
        'ratings'       : [],
        'most_recent_post' : []
    }
    
    print('GAME ' + str(game['name']))
    
    with open('game.json', 'a') as outfile:
        if w == 0:
            outfile.write(',')
        else:
            w -= 1
        json.dump(game, outfile)
    
    r = request('https://boardgamegeek.com/xmlapi2/forumlist?id=%i&type=thing' % (csv['id'][i],))
    
    if r == 0:
        continue
    
    soup = BeautifulSoup(r.text, "xml")
    
    tmp = soup.find_all('forum')
    forums = []
    
    for forum in tmp:
        
        r = request('https://boardgamegeek.com/xmlapi2/forum?id=%i' % (int(forum['id']),))
        
        if r == 0:
            continue
        
        soup = BeautifulSoup(r.text, "xml")
        
        threads = soup.find_all('thread')
        
        for thread in threads:
            print('\tTHREAD ' + str(thread['id']))
            
            r = request('https://boardgamegeek.com/xmlapi2/thread?id=%i' % (int(thread['id']),))
            
            if r == 0:
                continue
            
            soup = BeautifulSoup(r.text, "xml")
            
            post = {}
            comments = []
            likes = set()
            posts = soup.find_all('article')
            
            lenght = len(posts)
            
            for p in posts:
                try:
                    if lenght == len(posts):
                        post = {
                            'id'        : str(p['id']),
                            'author'    : p['username'],
                            'game'      : str(game['name']),
                            'subject'   : p.subject.string,
                            'text'      : p.body.string,
                            'timestamp' : p['postdate'],
                            'likes'     : [],
                            'comments'  : []
                        }
                        lenght -= 1
                    else:
                        comment = {
                            'author'    : p['username'],
                            'text'      : p.body.string,
                            'timestamp' : p['postdate']
                        }
                        comments.append(comment)
                        likes.add(p['username'])
                except:
                    print("\t\tSomething goes wrong. Skipping post/comment " + str(p['id']))
                    continue
            
            post['comments'] = comments
            post['likes'] = list(likes)
            
            with open('post.json', 'a') as outfile:
                if v == 0:
                    outfile.write(',')
                else:
                    v -= 1
                json.dump(post, outfile)

Page 1 scraped, minimum number of ratings was 27925
Page 2 scraped, minimum number of ratings was 18480
Page 3 scraped, minimum number of ratings was 13760
Page 4 scraped, minimum number of ratings was 10384
Page 5 scraped, minimum number of ratings was 8556
Page 6 scraped, minimum number of ratings was 7136
Page 7 scraped, minimum number of ratings was 6168
Page 8 scraped, minimum number of ratings was 5393
Page 9 scraped, minimum number of ratings was 4829
GAME Carcassonne
	THREAD 2938034
	THREAD 2797339
	THREAD 2887918
	THREAD 2734837
	THREAD 2815073
	THREAD 2812685
	THREAD 151931
	THREAD 2773410
	THREAD 2728079
	THREAD 2638718
	THREAD 2629025
	THREAD 2624923
	THREAD 2571757
	THREAD 2503991
	THREAD 2459428
	THREAD 2458273
	THREAD 2435663
	THREAD 2444022
	THREAD 2437613
	THREAD 2340536
	THREAD 2288531
	THREAD 2175033
	THREAD 2165799
	THREAD 2138931
	THREAD 2089010
	THREAD 2089006
	THREAD 2077306
	THREAD 2049241
	THREAD 2056548
	THREAD 539480
	THREAD 2006246
	THREAD 1964476
	THREAD 19

	THREAD 2590540
	THREAD 2665144
	THREAD 2700847
	THREAD 2700411
	THREAD 2697508
	THREAD 2613560
	THREAD 2656583
	THREAD 2649581
	THREAD 2647281
	THREAD 300287
	THREAD 2439355
	THREAD 2944897
	THREAD 2925252
	THREAD 295444
	THREAD 2890071
	THREAD 2836670
	THREAD 2797604
	THREAD 2790078
	THREAD 553694
	THREAD 2787440
	THREAD 2781942
	THREAD 2786293
	THREAD 2743541
	THREAD 2701284
	THREAD 336427
	THREAD 2341375
	THREAD 2605201
	THREAD 2605186
	THREAD 2566601
	THREAD 2318933
	THREAD 951085
	THREAD 2546460
	THREAD 982132
	THREAD 2543904
	THREAD 2518290
	THREAD 2530826
	THREAD 2516697
	THREAD 2515057
	THREAD 1181509
	THREAD 2492700
	THREAD 2489009
	THREAD 2461067
	THREAD 2455341
	THREAD 2443546
	THREAD 2442443
	THREAD 2442437
	THREAD 1464763
	THREAD 2437012
	THREAD 2432152
	THREAD 2403349
	THREAD 2400872
	THREAD 2396711
	THREAD 2302654
	THREAD 2384257
	THREAD 2356305
	THREAD 2360381
	THREAD 2175042
	THREAD 2309751
	THREAD 2356007
	THREAD 2996095
	THREAD 2873678
	THREAD 2826812
	THREAD 271058

	THREAD 2937582
	THREAD 2974602
	THREAD 2954374
	THREAD 150443
	THREAD 2914278
	THREAD 2246585
	THREAD 2904024
	THREAD 2900171
	THREAD 2896632
	THREAD 2885029
	THREAD 409082
	THREAD 2787207
	THREAD 2820000
	THREAD 1908576
	THREAD 2826113
	THREAD 2793386
	THREAD 2808797
	THREAD 429920
	THREAD 460453
	THREAD 2801564
	THREAD 2790782
	THREAD 2786893
	THREAD 2781788
	THREAD 2600940
	THREAD 2181130
	THREAD 2747974
	THREAD 2734464
	THREAD 2730753
	THREAD 2677585
	THREAD 2678299
	THREAD 2697071
	THREAD 123037
	THREAD 794913
	THREAD 2466204
	THREAD 1937275
	THREAD 964666
	THREAD 2551721
	THREAD 2170458
	THREAD 2628800
	THREAD 317843
	THREAD 2568359
	THREAD 2560252
	THREAD 2569218
	THREAD 2542691
	THREAD 2887876
	THREAD 2636080
	THREAD 2573265
	THREAD 2546978
	THREAD 2372772
	THREAD 1970012
	THREAD 1869248
	THREAD 450180
	THREAD 1793735
	THREAD 1705149
	THREAD 1705458
	THREAD 1580565
	THREAD 1557874
	THREAD 1502273
	THREAD 1502998
	THREAD 1302462
	THREAD 1355700
	THREAD 1323537
	THREAD 1302475
	

	THREAD 1520064
	THREAD 1467913
	THREAD 1425493
	THREAD 1424558
	THREAD 1346015
	THREAD 1329573
	THREAD 1328492
	THREAD 1328778
	THREAD 1323702
	THREAD 1323839
	THREAD 1306821
	THREAD 1277172
	THREAD 1251590
	THREAD 981912
	THREAD 537505
	THREAD 1045942
	THREAD 1035915
	THREAD 1030353
	THREAD 1029143
	THREAD 1017950
	THREAD 956244
	THREAD 938539
	THREAD 916174
	THREAD 904403
	THREAD 886236
	THREAD 889434
	THREAD 875554
	THREAD 874348
	THREAD 859556
	THREAD 856499
	THREAD 524457
	THREAD 763127
	THREAD 838975
	THREAD 819051
	THREAD 819277
	THREAD 810088
	THREAD 810875
	THREAD 850628
	THREAD 926575
	THREAD 2999261
	THREAD 2974916
	THREAD 3001279
	THREAD 2996330
	THREAD 2996549
	THREAD 2965403
	THREAD 2985703
	THREAD 2975680
	THREAD 2335481
	THREAD 2976649
	THREAD 2926519
	THREAD 2967627
	THREAD 640214
	THREAD 2926786
	THREAD 959801
	THREAD 2627493
	THREAD 2096313
	THREAD 386655
	THREAD 2882867
	THREAD 2954996
	THREAD 2952223
	THREAD 2876297
	THREAD 2845611
	THREAD 718823
	THREAD 2946498
	

	THREAD 2993136
	THREAD 1751612
	THREAD 2977836
	THREAD 2984350
	THREAD 2983294
	THREAD 1983193
	THREAD 2978694
	THREAD 2965594
	THREAD 2975998
	THREAD 2622885
	THREAD 2970314
	THREAD 2955732
	THREAD 2961822
	THREAD 1804448
	THREAD 2961425
	THREAD 1751530
	THREAD 2953594
	THREAD 2953561
	THREAD 2952508
	THREAD 2951568
	THREAD 2942193
	THREAD 2940991
	THREAD 1810908
	THREAD 2930666
	THREAD 2924775
	THREAD 2920975
	THREAD 2834464
	THREAD 2909447
	THREAD 2906643
	THREAD 2908457
	THREAD 1734106
	THREAD 1700292
	THREAD 2913341
	THREAD 2914141
	THREAD 2906176
	THREAD 2839452
	THREAD 2912366
	THREAD 2911291
	THREAD 2895695
	THREAD 2877214
	THREAD 2909327
	THREAD 2908406
	THREAD 2906167
	THREAD 2512860
	THREAD 2905905
	THREAD 2986510
	THREAD 2986792
	THREAD 2980021
	THREAD 2977528
	THREAD 2912761
	THREAD 2903627
	THREAD 2946841
	THREAD 1649367
	THREAD 2888111
	THREAD 2531846
	THREAD 2337102
	THREAD 2010718
	THREAD 2545408
	THREAD 2778629
	THREAD 2829012
	THREAD 1847708
	THREAD 2744874
	THREAD 

...

	THREAD 1608131
	THREAD 1650478
	THREAD 416695
	THREAD 1889154
	THREAD 1796073
	THREAD 659446
	THREAD 399122
	THREAD 1795528
	THREAD 1790173
	THREAD 1786443
	THREAD 1768403
	THREAD 746117
	THREAD 1716553
	THREAD 1716531
	THREAD 1487222
	THREAD 1657565
	THREAD 1634080
	THREAD 1609073
	THREAD 1553861
	THREAD 1569010
	THREAD 2840739
	THREAD 2666082
	THREAD 1834999
	THREAD 2170432
	THREAD 1660534
	THREAD 1563291
	THREAD 1770816
	THREAD 569704
	THREAD 1575095
	THREAD 1559004
	THREAD 1554951
	THREAD 1547430
	THREAD 1228925
	THREAD 1083237
	THREAD 1324094
	THREAD 1259356
	THREAD 1155825
	THREAD 692306
	THREAD 997153
	THREAD 868166
	THREAD 1068617
	THREAD 1064201
	THREAD 1056859
	THREAD 1000609
	THREAD 989094
	THREAD 1003131
	THREAD 952313
	THREAD 952338
	THREAD 701214
	THREAD 528196
	THREAD 334548
	THREAD 610911
	THREAD 760724
	THREAD 786777
	THREAD 726933
	THREAD 720170
	THREAD 720060
	THREAD 686472
	THREAD 684411
	THREAD 314552
	THREAD 580511
	THREAD 516735
	THREAD 640809
	THREAD 639106
	TH

	THREAD 2289843
	THREAD 2155206
	THREAD 2049721
	THREAD 1893113
	THREAD 1336469
	THREAD 1804607
	THREAD 1795015
	THREAD 1792236
	THREAD 1170486
An exception has occurred, probably a momentary loss of connection. Waiting one seconds...
	THREAD 1763288
	THREAD 1751239
An exception has occurred, probably a momentary loss of connection. Waiting one seconds...
	THREAD 1658517
An exception has occurred, probably a momentary loss of connection. Waiting one seconds...
	THREAD 1571254


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "E:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\feder\AppData\Local\Temp\ipykernel_3232\1820957067.py", line 199, in <module>
    r = request('https://boardgamegeek.com/xmlapi2/thread?id=%i' % (int(thread['id']),))
  File "C:\Users\feder\AppData\Local\Temp\ipykernel_3232\1820957067.py", line 24, in request
    sleep(slp)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101

TypeError: object of type 'NoneType' has no len()